In [1]:
import cv2
from enum import Enum
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt 
import matplotlib.patches as patches
import json
import os
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
import torch

In [34]:
cfg = get_cfg()
cfg.merge_from_file("../configs/quick_schedules/mask_rcnn_R_50_FPN_inference_acc_test.yaml") 

predictor = DefaultPredictor(cfg)


def detect_mask(np_image,threshold=0.30):
    predictions = predictor(np_image)
    visualizer = Visualizer(np_image)

    instances = predictions["instances"].to(torch.device("cpu"))
    instances = instances[instances.pred_classes==2]
    if len(instances) > 0:
        filtered_instances = instances[instances.scores > threshold]

        if len(filtered_instances) == 0:
            filtered_instances = instances[instances.scores == instances.scores.max()]
    else:
        filtered_instances = instances

    vis_output = visualizer.draw_instance_predictions(predictions=filtered_instances)
    return vis_output.get_image(),filtered_instances

In [58]:
image_folder = "../images/Flood/"
label_folder = "../images/labels/"
mask_result_folder = "../result_mask/"
file_names = os.listdir(image_folder)
# image_name = "Flood_27.jpg"  # "Flood_183.jpg" flip
for image_name in file_names:
    base_name = os.path.splitext(image_name)[0]
    label_name = base_name+".json"

    pil_image = Image.open(image_folder+image_name)
    im_width, im_height = pil_image.size
    np_image = np.array(pil_image)

    result,instances = detect_mask(np_image,0.30)

    Image.fromarray(result).save(mask_result_folder+image_name)

    data = {"cars":[]}
    
    for i in range(0,len(instances)):
        box = instances.pred_boxes[i].tensor.numpy()[0]
        mask = np.zeros((im_height,im_width))
        mask[instances.pred_masks[i].numpy()] = 1
        data["cars"].append({"box":box.tolist(),"mask":mask.tolist()})

    with open(mask_result_folder+base_name+".json","w") as file:
        file.write(json.dumps(data))

In [98]:
image_folder = "../images/Flood/"
image_name = "Flood_1457.jpg"

pil_image = Image.open(image_folder+image_name)
im_width, im_height = pil_image.size
np_image = np.array(pil_image)

predictions = predictor(np_image)
visualizer = Visualizer(np_image)

instances = predictions["instances"].to(torch.device("cpu"))
instances = instances[instances.pred_classes==2]
if len(instances) > 0:
    filtered_instances = instances[instances.scores > 0.91]
    # filtered_instances = filtered_instances[filtered_instances.scores < 0.20]

    if len(filtered_instances) == 0:
        filtered_instances = instances[instances.scores == instances.scores.max()]
else:
    filtered_instances = instances

vis_output = visualizer.draw_instance_predictions(predictions=filtered_instances)


base_name = os.path.splitext(image_name)[0]
data = {"cars":[]}
for i in range(0,len(filtered_instances)):
    box = filtered_instances.pred_boxes[i].tensor.numpy()[0]
    mask = np.zeros((im_height,im_width))
    mask[filtered_instances.pred_masks[i].numpy()] = 1
    data["cars"].append({"box":box.tolist(),"mask":mask.tolist()})

with open(mask_result_folder+base_name+".json","w") as file:
    file.write(json.dumps(data))

Image.fromarray(vis_output.get_image()).save(mask_result_folder+image_name)